##Evaluation Pipeline Overview
Following the training phase, the rigorous evaluation of the model is handled by this script. The objective is not only to rely on high-level metrics but to analyze the raw outputs and manually verify the intersection over union (IoU). This ensures it is understood exactly how well stenotic lesions are localized by the model.

In [ ]:
%pip install opencv-python-headless matplotlib

print("Installation complete.")

You should consider upgrading via the '/Users/mariaplatek/projects/AXION/YOLOv11/.venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/mariaplatek/projects/AXION/YOLOv11/.venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
✅ Installation complete.


In [ ]:
import torch
from ultralytics import YOLO
import os
import glob

import numpy as np

In [ ]:
if torch.cuda.is_available():
    print("NVIDIA GPU (CUDA) detected.")
    device = "cuda:0"
elif torch.backends.mps.is_available():
    print("Apple Silicon (MPS) detected!")
    device = "mps"
else:
    print("Using CPU.")
    device = "cpu"

Apple Silicon (MPS) detected!


###Hardware Initialization & Standard Validation
The best.pt weights generated from the training phase are loaded, and the standard YOLO validation method is executed against the unseen test split. A reliable baseline of the model's performance is thereby established, outputting the standard Mean Average Precision (mAP@50 and mAP@50-95), Precision, Recall and F1 scores.

In [ ]:
model = YOLO("weights/best.pt")

print(f"\nRunning Evaluation on Test Set using {device}...")
metrics = model.val(
    data="dataset/data.yaml",
    split='test',
    device=device,
    plots=True
)

print("\n--- FINAL METRICS ---")
print(f"mAP@50:    {metrics.box.map50:.2%}")
print(f"mAP@50-95: {metrics.box.map:.2%}")
print(f"Precision: {metrics.box.mp:.2%}")
print(f"Recall:    {metrics.box.mr:.2%}")

f1_mean = np.mean(metrics.box.f1)
print(f"F1 Score: {f1_mean:.2%}")


Running Evaluation on Test Set using mps...
Ultralytics 8.4.14 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M4 Pro)
YOLO26s summary (fused): 122 layers, 9,465,567 parameters, 0 gradients, 20.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 255.5±75.5 MB/s, size: 91.4 KB)
val: Scanning /Users/mariaplatek/projects/AXION/YOLOv11/YOLO26s/dataset/test/labels... 821 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 821/821 5.3Kit/s 0.2s0.2s
val: New cache created: /Users/mariaplatek/projects/AXION/YOLOv11/YOLO26s/dataset/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 52/52 3.6it/s 14.3s0.3s
                   all        821        821       0.96      0.936      0.972      0.495
Speed: 0.1ms preprocess, 8.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /Users/mariaplatek/projects/AXION/YOLOv11/YOLO26s/runs/detect/val

--- FINAL METRICS ---
mAP@50:    97.22%
mAP@50-95: 49.52%
Precision:

In [ ]:
images_dir = "dataset/test/images"
labels_dir = "dataset/test/labels"

###Dynamic Sample Prediction
To verify the practical functionality of the model, a dynamic prediction block is implemented. The first available .jpg image from the test set is automatically retrieved, and a live inference is run with a confidence threshold of 0.5.

**Coordinates (xyxy):** The exact pixel locations of the bounding box corners are displayed.
These represent the exact pixel locations of the bounding box drawn around the detected object.
The format is `[x_min, y_min, x_max, y_max]`.

**Confidence Scores:** The certainty that the detected object is a stenosis is evaluated and printed.

**Class IDs:** The correct label category prediction is verified.

In [ ]:
# --- Dynamic Prediction ---
model.to(device)

# Grab the first available image in the test directory dynamically
test_images = glob.glob(os.path.join(images_dir, "*.jpg"))

if test_images:
    sample_image = test_images[0]
    print(f"\nRunning prediction on sample: {sample_image}")

    results = model.predict(
        sample_image,
        conf=0.5,
        device=device,
        half=False
    )

    for r in results:
        boxes = r.boxes
        print("\n--- RAW MODEL OUTPUT ---")
        print(f"Coordinates (xyxy): {boxes.xyxy.tolist()}")
        print(f"Confidence scores: {boxes.conf.tolist()}")
        print(f"Class IDs: {boxes.cls.tolist()}")
else:
    print(f"Warning: No .jpg images found in {images_dir}!")


Running prediction on sample: dataset/test/images/14_031_5_0036_bmp.rf.97d1144ba125963340270c5b4f390754.jpg

image 1/1 /Users/mariaplatek/projects/AXION/YOLOv11/YOLO26s/dataset/test/images/14_031_5_0036_bmp.rf.97d1144ba125963340270c5b4f390754.jpg: 640x640 1 Stenosis, 14.5ms
Speed: 4.1ms preprocess, 14.5ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)

--- RAW MODEL OUTPUT ---
Coordinates (xyxy): [[488.6090087890625, 191.39208984375, 525.1990966796875, 226.6587677001953]]
Confidence scores: [0.8094199299812317]
Class IDs: [0.0]


###Custom Intersection over Union (IoU) Engine
While standard metrics are utilized, granular control over how bounding box accuracy is calculated was required. To achieve this, a custom mathematical function was developed to calculate the Intersection over Union (IoU) using True Positives (TP), False Positives (FP), and False Negatives (FN) based on box areas.

In [ ]:
def calculate_single_iou_by_area(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])

    # 1. Calculate TP (True Positive area) -> The Intersection
    TP = max(0, xB - xA) * max(0, yB - yA)

    # Calculate the total area of each individual box
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]) # Predicted Box
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1]) # Ground Truth Box

    # Calculate FP (False Positive area) -> Predicted area minus Intersection
    FP = boxAArea - TP

    # Calculate FN (False Negative area) -> Ground Truth area minus Intersection
    FN = boxBArea - TP

    # IoU = TP / (TP + FP + FN)
    denominator = TP + FP + FN

    if denominator == 0:
        return 0.0

    return TP / denominator

def get_average_iou(model, images_dir, labels_dir):
    print("\nCalculating Average IoU using (TP / TP+FP+FN)...")
    total_iou = 0
    true_positives = 0

    image_paths = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_paths.extend(glob.glob(os.path.join(images_dir, ext)))

    for img_path in image_paths:
        # prediction
        results = model(img_path, device=device, verbose=False)
        pred_boxes = results[0].boxes.xyxy.cpu().numpy()
        img_h, img_w = results[0].orig_shape

        filename = os.path.basename(img_path)
        label_filename = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_filename)

        if not os.path.exists(label_path):
            continue

        gt_boxes = []
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    _, cx_n, cy_n, w_n, h_n = map(float, parts[:5])
                    cx, cy = cx_n * img_w, cy_n * img_h
                    w, h = w_n * img_w, h_n * img_h
                    x1, y1 = cx - w/2, cy - h/2
                    x2, y2 = cx + w/2, cy + h/2
                    gt_boxes.append([x1, y1, x2, y2])

        for p_box in pred_boxes:
            best_iou = 0
            for g_box in gt_boxes:
                # Use the new explicit TP/FP/FN function
                iou = calculate_single_iou_by_area(p_box, g_box)
                if iou > best_iou:
                    best_iou = iou

            # If overlap is 50% or higher, count it as a True Positive
            if best_iou >= 0.50:
                total_iou += best_iou
                true_positives += 1

    if true_positives > 0:
        avg_iou = total_iou / true_positives
        print(f"\n--> Evaluated {true_positives} True Positive detections.")
        print(f"--> Average Box Overlap (IoU): {avg_iou*100:.2f}%")
    else:
        print("\n--> No valid detections found to calculate IoU.")

get_average_iou(model, images_dir, labels_dir)


Calculating Average IoU using (TP / TP+FP+FN)...

--> Evaluated 788 True Positive detections.
--> Average Box Overlap (IoU): 75.04%
